In [1]:
# Dependencies
from splinter import Browser
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
import requests
import pymongo
from flask import Flask, render_template, redirect
from flask_pymongo import PyMongo
import time

In [2]:
!which chromedriver

/usr/local/bin/chromedriver


In [3]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

# NASA Mars News

In [7]:
# Create a dictionary to hold everything pulled from all the sites
scraped_data = {}

In [9]:
url = "https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest"
browser.visit(url)

# Create a Beautiful Soup object
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [10]:
# Get the news title and store it in the dictionnary
news_title = soup.find('div', class_='content_title').text
scraped_data['news_title'] = news_title
print(scraped_data['news_title'])

# Get the news text and store it in the dictionnary
news_p = soup.find('div', class_='article_teaser_body').text
scraped_data['news_p'] = news_p
print(scraped_data['news_p'])

Mars Now
The small craft will seek to prove that powered, controlled flight is possible on another planet. But just getting it onto the surface of Mars will take a whole lot of ingenuity.


In [12]:
news_list = soup.find('ul', class_='item_list')
first_item = news_list.find('li', class_='slide')
news_headline = first_item.find('div', class_='content_title').text
news_teaser = first_item.find('div', class_='article_teaser_body').text
print(news_headline)
print(news_teaser)

How NASA's Mars Helicopter Will Reach the Red Planet's Surface
The small craft will seek to prove that powered, controlled flight is possible on another planet. But just getting it onto the surface of Mars will take a whole lot of ingenuity.


In [13]:
scraped_data['news_title'] = news_headline
print(scraped_data['news_title'])

scraped_data['news_body'] = news_teaser
print(scraped_data['news_body'])

How NASA's Mars Helicopter Will Reach the Red Planet's Surface
The small craft will seek to prove that powered, controlled flight is possible on another planet. But just getting it onto the surface of Mars will take a whole lot of ingenuity.


# JPL Mars Space Images - Featured Image

In [32]:
jpl_url = "https://www.jpl.nasa.gov/spaceimages/"
browser.visit(jpl_url)

In [36]:
browser.links.find_by_partial_text('FULL IMAGE')
time.sleep(1)

In [37]:
jpl_html = browser.html
jpl_soup = BeautifulSoup(jpl_html, 'html.parser')

In [38]:
jpl_img = jpl_soup.find('img', class_='fancybox-image')['src']
img_path = f'https://www.jpl.nasa.gov{jpl_img}'
print(img_path)

https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA20063_ip.jpg


# Mars Weather

In [9]:
weather_url = "https://twitter.com/marswxreport?lang=en"
browser.visit(weather_url)

In [10]:
w_html = browser.html
wsoup = BeautifulSoup(w_html,'lxml')

In [11]:
tweets = wsoup.find_all('p',class_='TweetTextSize')
for tweet in tweets:
    tweet_text = tweet.text
    if tweet_text.startswith('Sol'):
        mars_weather = tweet_text
        break
    else:
        continue

In [27]:
from selenium import webdriver
driver = webdriver.Chrome()
driver.get(twitter_url)
html = driver.page_source

twitter_url = "https://twitter.com/marswxreport?lang=en"
soup = BeautifulSoup(html, "html.parser")

mars_weather = soup.find("<span", class_="css-901oao css-16my406 r-1qd0xha r-ad9z0x r-bcqeeo r-qvutc0")

driver.close()

print(mars_weather)

None


In [11]:
# Dependencies
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup

def init_browser():
    executable_path = {"executable_path": "/usr/local/bin/chromedriver"}
    return Browser("chrome", **executable_path, headless=False)

browser = init_browser()

# Mars Weather twitter 
weather_url = "https://twitter.com/marswxreport?lang=en"
browser.visit(weather_url)

# HTML object
html = browser.html

# Parse HTML w/ Beautiful Soup
weather_soup = BeautifulSoup(html, "html.parser")

In [12]:
# Find all elements that contain tweets
latest_tweets = weather_soup.find_all('div', class_='css-901aoa ')

# Retrieve all elements that contain news title in the specified range & look weather related words
for tweet in latest_tweets: 
    mars_weather = tweet.find('p').text
    if 'Sol' and 'pressure' in mars_weather:
        print(mars_weather)
        break
    else: 
        pass

# Mars Facts

In [30]:
import pandas as pd

facts_url = "https://space-facts.com/mars/"
tables = pd.read_html(facts_url)

In [31]:
facts_df = tables[0]
facts_df.columns = ["Fact","Value"]

# get rid of trailing colon
facts_df["Fact"] = facts_df["Fact"].str[:-1]
facts_df = facts_df.set_index("Fact")
facts_df

,Value
Fact,
Equatorial Diameter,"6,792 km"
Polar Diameter,"6,752 km"
Mass,6.39 × 10^23 kg (0.11 Earths)
Moons,2 (Phobos & Deimos)
Orbit Distance,"227,943,824 km (1.38 AU)"
Orbit Period,687 days (1.9 years)
Surface Temperature,-87 to -5 °C
First Record,2nd millennium BC
Recorded By,Egyptian astronomers


In [32]:
facts_html_table = facts_df.to_html()
facts_html_table = facts_html_table.replace('\n', '')
facts_html_table

'<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>Value</th>    </tr>    <tr>      <th>Fact</th>      <th></th>    </tr>  </thead>  <tbody>    <tr>      <th>Equatorial Diameter</th>      <td>6,792 km</td>    </tr>    <tr>      <th>Polar Diameter</th>      <td>6,752 km</td>    </tr>    <tr>      <th>Mass</th>      <td>6.39 × 10^23 kg (0.11 Earths)</td>    </tr>    <tr>      <th>Moons</th>      <td>2 (Phobos &amp; Deimos)</td>    </tr>    <tr>      <th>Orbit Distance</th>      <td>227,943,824 km (1.38 AU)</td>    </tr>    <tr>      <th>Orbit Period</th>      <td>687 days (1.9 years)</td>    </tr>    <tr>      <th>Surface Temperature</th>      <td>-87 to -5 °C</td>    </tr>    <tr>      <th>First Record</th>      <td>2nd millennium BC</td>    </tr>    <tr>      <th>Recorded By</th>      <td>Egyptian astronomers</td>    </tr>  </tbody></table>'

# Mars hemispheres photo scraping

In [33]:
from bs4 import BeautifulSoup
from splinter import Browser

executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

base_hemisphere_url = "https://astrogeology.usgs.gov"
hemisphere_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(hemisphere_url)

In [34]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
    
hemisphere_image_urls = []

links = soup.find_all("div", class_="item")

for link in links:
    img_dict = {}
    title = link.find("h3").text
    next_link = link.find("div", class_="description").a["href"]
    full_next_link = base_hemisphere_url + next_link
    
    browser.visit(full_next_link)
    
    pic_html = browser.html
    pic_soup = BeautifulSoup(pic_html, 'html.parser')
    
    url = pic_soup.find("img", class_="wide-image")["src"]

    img_dict["title"] = title
    img_dict["img_url"] = base_hemisphere_url + url
    print(img_dict["img_url"])
    
    hemisphere_image_urls.append(img_dict)

https://astrogeology.usgs.gov/cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg
https://astrogeology.usgs.gov/cache/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg
https://astrogeology.usgs.gov/cache/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg
https://astrogeology.usgs.gov/cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg


In [35]:
hemisphere_image_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]